In [74]:
!nvidia-smi

Tue Mar 28 14:41:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    34W /  70W |   2221MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Importing Libraries

In [128]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import os
import urllib
import zipfile
from tqdm.auto import tqdm


# GPU Config

In [77]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# HyperParameter

In [121]:
batch_size = 64
input_channel = 3 # RGB
n_filters = 12
kernel_size = 3 # filter size of 3*3
output = 200
n_iters = 5
learning_rate = 0.001
temperature = 4
alpha = 0.999
decay=0.999

# Download and Extract Dataset

In [79]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

!unzip -qq 'tiny-imagenet-200.zip'

--2023-03-28 14:41:28--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip.1’

tiny-imagenet-200.z  37%[======>             ]  88.08M  15.2MB/s    eta 17s    ^C
replace tiny-imagenet-200/words.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Data Folder path

In [80]:
data_dir = '/content/tiny-imagenet-200'

# Define training and validation data paths
train_dir = os.path.join(data_dir, 'train') 
test_dir = os.path.join(data_dir, 'test') 
val_dir = os.path.join(data_dir, 'val')


# Dataset and DataLoader

In [81]:
train_dataset = torchvision.datasets.ImageFolder(train_dir,transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)


test_dataset = torchvision.datasets.ImageFolder(test_dir,transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)


val_dataset = torchvision.datasets.ImageFolder(val_dir,transform=transforms.ToTensor())
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size)

# Loader Varification

In [82]:
image, label = next(iter(train_loader))
print(image.shape)
print(label.shape)

torch.Size([64, 3, 64, 64])
torch.Size([64])


# teacher Implementation

In [84]:
class teacherNet(nn.Module):
  def __init__(self):
    super(teacherNet, self).__init__()
    self.cv1 = nn.Conv2d(in_channels= 3, out_channels= 12, kernel_size= 5, padding= 1, stride=1)
    self.cv2 = nn.Conv2d(in_channels= 12, out_channels = 24, kernel_size= 5, padding=1, stride = 1)
    self.cv3 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv4 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv5 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv6 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv7 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv8 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv9 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv10 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv11 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.cv12 = nn.Conv2d(in_channels= 24, out_channels= 24, kernel_size=5, padding=1, stride=1)
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
    self.tanh = nn.Tanh()
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(24*20*20, 1024)
    self.fc2 = nn.Linear(1024, 200)


    # xavier weight initialization
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight).to(device)

      elif isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight).to(device)

      
  def forward(self,x):
    x = self.cv1(x)
    x = self.tanh(x)
    # print("cnv 1",x.shape)

    x = self.cv2(x)
    x = self.tanh(x)
    # print("cnv 2",x.shape)

    x = self.cv3(x)
    x = self.tanh(x)
    # print("cnv 3",x.shape)

    x = self.cv4(x)
    x = self.tanh(x)
    # print("cnv 4",x.shape)

    x = self.cv5(x)
    x = self.tanh(x)
    # print("cnv 5",x.shape)

    x = self.cv6(x)
    x = self.tanh(x)
    # print("cnv 6",x.shape)

    x = self.cv7(x)
    x = self.tanh(x)
    # print("cnv 7",x.shape)

    x = self.cv8(x)
    x = self.tanh(x)
    # print("cnv 8",x.shape)

    x = self.cv9(x)
    x = self.tanh(x)
    # print("cnv 9",x.shape)

    x = self.cv10(x)
    x = self.tanh(x)
    # print("cnv 10",x.shape)

    x = self.cv11(x)
    x = self.tanh(x)
    # print("cnv 11",x.shape)

    x = self.cv12(x)
    x = self.tanh(x)
    x = self.pool(x)
    # print("cnv 12",x.shape)

    # x = x.view(x.size(0), -1)
    x = self.flat(x)
    x = self.fc1(x)
    x = self.tanh(x)
    # print("Fc1 : ",x.shape)

    x = self.fc2(x)

    # print(x.shape)
    return x    



teacher_model = teacherNet()
teacher_model.to(device)
teacher_model


teacherNet(
  (cv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv3): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv4): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv5): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv6): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv7): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv8): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv9): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv10): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv11): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv12): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (tanh): Tanh()
  (flat): Fla

# Student Model implementation

In [85]:
class studentNet(nn.Module):
  def __init__(self):
    super(studentNet, self).__init__()
    self.cv1 = nn.Conv2d(in_channels = 3, out_channels=12, kernel_size = 5, stride=1, padding=1)
    self.cv2 = nn.Conv2d(in_channels = 12, out_channels=24, kernel_size = 5, stride=1, padding=1)
    self.cv3 = nn.Conv2d(in_channels = 24, out_channels=24, kernel_size = 5, stride=1, padding=1)
    self.cv4 = nn.Conv2d(in_channels = 24, out_channels=24, kernel_size = 5, stride=1, padding=1)
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
    self.tanh = nn.Tanh()
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(24*28*28, 1024)
    self.fc2 = nn.Linear(1024, 200)


    # xavier weight initialization
    for m in self.modules():
        if isinstance(m, nn.Conv2d):
          nn.init.xavier_uniform_(m.weight).to(device)

        elif isinstance(m, nn.Linear):
          nn.init.xavier_uniform_(m.weight).to(device)

  def forward(self, x):
    x = self.cv1(x)
    x = self.tanh(x)
    # print("cnv 1",x.shape)

    x = self.cv2(x)
    x = self.tanh(x)
    # print("cnv 2",x.shape)

    x = self.cv3(x)
    x = self.tanh(x)
    # print("cnv 3",x.shape)

    x = self.cv4(x)
    x = self.tanh(x)
    x = self.pool(x)
    # print("cnv 12",x.shape)

    # x = x.view(x.size(0), -1)
    x = self.flat(x)
    x = self.fc1(x)
    x = self.tanh(x)
    # print("Fc1 : ",x.shape)

    x = self.fc2(x)
    # print("Fc2 : ",x.shape)
    return x    

student_model = studentNet().to(device)
student_model

studentNet(
  (cv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv3): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (cv4): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (tanh): Tanh()
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=18816, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=200, bias=True)
)

# xaviour weight initialization

In [86]:
# nn.init.xavier_uniform_(model.conv1.weight)
# nn.init.xavier_uniform_(model.conv2.weight)
# nn.init.xavier_uniform_(model.conv3.weight)
# nn.init.xavier_uniform_(model.conv4.weight)
# nn.init.xavier_uniform_(model.conv5.weight)
# nn.init.xavier_uniform_(model.conv6.weight)
# nn.init.xavier_uniform_(model.fc1.weight)
# nn.init.xavier_uniform_(model.fc2.weight)

In [87]:
# def weight_init(m):
#   if isinstance(m, nn.Conv2d):
#     nn.init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain('tanh'))

# model2 = ConvNet()
# model2.apply(weight_init)

In [88]:
# print(model2.conv1.weight)
# print(model2.conv2.weight)
# print(model2.conv3.weight)
# print(model2.conv4.weight)
# print(model2.conv5.weight)
# print(model2.conv6.weight)  

# Loss and Optimizer

In [89]:
loss_fun = nn.CrossEntropyLoss()
loss_fun

CrossEntropyLoss()

In [90]:
optimizer_teacher = torch.optim.SGD(teacher_model.parameters(), lr= learning_rate)
optimizer_teacher

optimizer_student = torch.optim.SGD(student_model.parameters(), lr= learning_rate)
optimizer_student

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

# Graph Plot Code

In [134]:
epoch = []
no_of_epochs=n_iters
for i in range(no_of_epochs):
  epoch.append(i+1)

def plot_graph(y):
  plt.plot(epoch,y)
  plt.show() #display the graph

# Model Without EMA

## Training Loop for teacher without EMA

In [132]:
total_steps = len(train_loader)
train_loss_list = []
train_acc_list = []
test_acc_list = []
for epoch in tqdm(range(n_iters)):
  total_samples=0
  correct_samples=0
  
  for step, (image, label) in enumerate(train_loader):
    image = image.to(device)
    label = label.to(device)

    output = teacher_model(image)
    # print(output.shape)
    # print(label.shape)
    # loss
    loss = loss_fun(output, label)


    _,out=torch.max(output,1)
    total_samples+=label.shape[0]
    correct_samples+=(out==label).sum().item()
    
    #backward pass
    optimizer_teacher.zero_grad()
    loss.backward()
    optimizer_teacher.step()


  accuracy=(correct_samples/total_samples)*100.0
  print("Epoch: {}/{} | step:{}/{} | Loss:{:.4f} | Accuracy{: .2f}".format(epoch,n_iters,step,total_steps,loss.item(),accuracy))
  train_loss_list.append(loss.item())
  train_acc_list.append(accuracy)

  # Test Loop 
  with torch.no_grad():
    n_correct = 0
    n_samples = 0

    for image, label in test_loader:
      image = image.to(device)
      label = label.to(device)

      output = teacher_model(image)

      # value, index
      _, predictions = torch.max(output, 1)
      n_samples += label.shape[0]
      n_correct += (predictions == label).sum().item()
      # print(n_correct)


    acc = 100.0 * (n_correct/n_samples)
    test_acc_list.append(acc)
  print("Accuracy : ",acc)

    
# print("\n\nloss list",list)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0/5 | step:1562/1563 | Loss:0.0909 | Accuracy 71.89
Accuracy :  0.0
Epoch: 1/5 | step:1562/1563 | Loss:0.0681 | Accuracy 72.91
Accuracy :  0.0
Epoch: 2/5 | step:1562/1563 | Loss:0.0812 | Accuracy 73.82
Accuracy :  0.0


KeyboardInterrupt: ignored

In [ ]:
plot_graph(train_loss_list)
plot_graph(train_acc_list)
plot_graph(test_acc_list)


## student model without EMA

In [ ]:
import torch.nn.functional as F
scheduler = StepLR(optimizer_student, step_size=30, gamma=0.1)

total_steps = len(train_loader)
loss_list = []
acc_list = []
test_loss_list = []
test_acc_list = []  
T = 3
for epoch in tqdm(range(n_iters)):
  total_samples=0
  correct_samples=0
  running_loss = 0.0
  for step, (image, label) in enumerate(train_loader):
    image = image.to(device)
    label = label.to(device)
    optimizer_teacher.zero_grad()


    with torch.no_grad():
      teacher_pred = teacher_model(image)
    # print(output.shape)
    # print(label.shape)
    # loss
    student_pred = student_model(image)

    # print(type(student_pred))
    # print(type(teacher_pred))
    student_loss = loss_fun(student_pred, label)


    # dest_loss = destillation_loss(student_pred, teacher_pred)
    soft_teacher_pred = F.softmax(teacher_pred / temperature, dim=1)
    # print("Soft teacher : ", soft_teacher_pred)
    soft_student_pred = F.softmax(student_pred / temperature, dim=1)
    # print("Soft student : ", soft_student_pred)
    # dest_loss = F.kl_div(soft_teacher_pred.log(), soft_student_pred, reduction='batchmean') * temperature * temperature

    dest_loss = nn.KLDivLoss()(F.log_softmax(soft_teacher_pred/T, dim=1),F.softmax(soft_student_pred/T, dim=1)) * T * T 
    # print(dest_loss)


    # print(dest_loss, type(dest_loss))

    loss = alpha * student_loss + (1 - alpha) * dest_loss

    loss.backward()

    # Update parameters
    for param in student_model.parameters():
        param.grad *= (temperature ** 2)


    optimizer_student.step()

    _,out=torch.max(student_pred,1)
    total_samples+=label.shape[0]
    correct_samples+=(out==label).sum().item()
    # break

  accuracy=(correct_samples/total_samples)*100.0
  print("Epoch: {}/{} | step:{}/{} | Loss:{:.4f} | Accuracy{: .2f}".format(epoch,n_iters,step,total_steps,loss.item(),accuracy))
  loss_list.append(loss.item())
  
  # update learning rate
  scheduler.step()
  # validate model
  student_model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = student_model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  acc = 100 * correct / total
  print('Test : Epoch %d: Loss = %.3f, Test Accuracy = %.3f %%' %
        (epoch + 1, running_loss / len(train_loader), acc))
  test_acc_list.append(acc)
  test_loss_list.append(running_loss)

    
# print("\n\nloss list",list)

In [ ]:
plot_graph(loss_list)
plot_graph(acc_list)
plot_graph(test_loss_list)
plot_graph(test_acc_list)


In [ ]:
criterion = nn.KLDivLoss()
optimizer = torch.optim.SGD(student_model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# Model With EMA

In [ ]:
# Define the EMA model
class EMA:
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                param.data = self.shadow[name]


## Teacher model with EMA

In [ ]:
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []
total_steps = len(train_loader)
ema = EMA(decay = decay, model = teacher_model)
ema.register()
for epoch in tqdm(range(n_iters)):
  total_samples=0
  correct_samples=0
  
  for step, (image, label) in enumerate(train_loader):
    image = image.to(device)
    label = label.to(device)

    output = teacher_model(image)
    loss = loss_fun(output, label)


    _,out=torch.max(output,1)
    total_samples+=label.shape[0]
    correct_samples+=(out==label).sum().item()
    
    #backward pass
    optimizer_teacher.zero_grad()
    loss.backward()
    optimizer_teacher.step()
    ema.update()


  accuracy=(correct_samples/total_samples)*100.0
  print("Epoch: {}/{} | step:{}/{} | Loss:{:.4f} | Accuracy{: .2f}".format(epoch,n_iters,step,total_steps,loss.item(),accuracy))
  loss_list.append(loss.item())

  scheduler.step()
  # Test Loop 
  with torch.no_grad():
    n_correct = 0
    n_samples = 0

    for image, label in test_loader:
      image = image.to(device)
      label = label.to(device)

      output = teacher_model(image)

      # value, index
      _, predictions = torch.max(output, 1)
      n_samples += label.shape[0]
      n_correct += (predictions == label).sum().item()
      # print(n_correct)


    acc = 100.0 * (n_correct/n_samples)

print("Accuracy : ",acc)

    
# print("\n\nloss list",list)

In [ ]:
plot_graph(train_loss_list)
plot_graph(train_acc_list)
plot_graph(test_acc_list)


## Student model with EMA

In [ ]:
import torch.nn.functional as F
scheduler = StepLR(optimizer_student, step_size=30, gamma=0.1)

total_steps = len(train_loader)
loss_list = []
acc_list = []
test_loss_list = []
test_acc_list = []  
T = 3

ema = EMA(decay = decay, model = student_model)
ema.register()


for epoch in tqdm(range(n_iters)):
  total_samples=0
  correct_samples=0
  running_loss = 0.0
  for step, (image, label) in enumerate(train_loader):
    image = image.to(device)
    label = label.to(device)
    optimizer_teacher.zero_grad()


    with torch.no_grad():
      teacher_pred = teacher_model(image)
    # print(output.shape)
    # print(label.shape)
    # loss
    student_pred = student_model(image)

    # print(type(student_pred))
    # print(type(teacher_pred))
    student_loss = loss_fun(student_pred, label)


    # dest_loss = destillation_loss(student_pred, teacher_pred)
    soft_teacher_pred = F.softmax(teacher_pred / temperature, dim=1)
    # print("Soft teacher : ", soft_teacher_pred)
    soft_student_pred = F.softmax(student_pred / temperature, dim=1)
    # print("Soft student : ", soft_student_pred)
    # dest_loss = F.kl_div(soft_teacher_pred.log(), soft_student_pred, reduction='batchmean') * temperature * temperature

    dest_loss = nn.KLDivLoss()(F.log_softmax(soft_teacher_pred/T, dim=1),F.softmax(soft_student_pred/T, dim=1)) * T * T 
    # print(dest_loss)


    # print(dest_loss, type(dest_loss))

    loss = alpha * student_loss + (1 - alpha) * dest_loss

    loss.backward()

    # Update parameters
    for param in student_model.parameters():
        param.grad *= (temperature ** 2)


    optimizer_student.step()
    ema.update()
    _,out=torch.max(student_pred,1)
    total_samples+=label.shape[0]
    correct_samples+=(out==label).sum().item()
    # break

  accuracy=(correct_samples/total_samples)*100.0
  print("Train Epoch: {}/{} | step:{}/{} | Loss:{:.4f} | Accuracy{: .2f}".format(epoch,n_iters,step,total_steps,loss.item(),accuracy))
  loss_list.append(loss.item())
  
  # update learning rate
  scheduler.step()
  # validate model
  student_model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = student_model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  acc = 100 * correct / total
  print("Test Epoch: {}/{} | step:{}/{} | Loss:{:.4f} | Accuracy{: .2f}".format(epoch,n_iters,step,total_steps,loss.item(),acc))
  test_acc_list.append(acc)
  test_loss_list.append(running_loss)

    
# print("\n\nloss list",list)

In [ ]:
plot_graph(loss_list)
plot_graph(acc_list)
plot_graph(test_loss_list)
plot_graph(test_acc_list)
